## Conolutional Neural Network to execute the OCR:
!!! Note it is assumed that for training and testing purposes the images have already been preprocessed with the help of Module 2 !!!

In [16]:
# Packages Required :
import cv2
import numpy as np
import glob
import os
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from matplotlib import pyplot as plt
from PIL import Image

### Defining the model:
The model's hyperparameters, the input shape and the layers have been defined in the following 
code snippet:

In [17]:
#Model definition :

# Hyper_parameters:
num_classes = 6
epochs = 10

# input image dimensions :
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

# Model Layers :
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

### Inputing the Images:
In the following code snippet, provide the path to the directory where the images have been classified ie. the images of a given digit have been put into a single folder with the name of the directory as the digit.
The snippet extracts the images from each digit's directory, label's the according to the directory they are in, and then shuffles all the images to provide us with an object of type DirectoryIterator which contains tuples (X, y) : the training images and the corresponding labels.

In [18]:
# To extract the images and label them according to their directory:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0,
        zoom_range=0,
        horizontal_flip=False,
        width_shift_range=0.,  
        height_shift_range=0.) 

# *** Provide the path to the directory with the digit classes here :
path_to_read = '/home/ignitarium/Desktop/Text_seg/classes/'
train_generator = train_datagen.flow_from_directory(
    directory = path_to_read,
    target_size = (28, 28),
    color_mode = 'grayscale',
    class_mode = 'categorical',
    shuffle = True,
    seed = 42
)

### Train the model:

In [20]:
# Training the model:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=30)

Epoch 1/30
8/8 [==============================] - 2s 233ms/step - loss: 1.7793 - acc: 0.2109
Epoch 2/30
8/8 [==============================] - 1s 125ms/step - loss: 1.6624 - acc: 0.3759
Epoch 3/30
8/8 [==============================] - 1s 124ms/step - loss: 1.5372 - acc: 0.3722
Epoch 4/30
8/8 [==============================] - 1s 126ms/step - loss: 1.3066 - acc: 0.4348
Epoch 5/30
8/8 [==============================] - 1s 125ms/step - loss: 1.3156 - acc: 0.4311
Epoch 6/30
8/8 [==============================] - 1s 125ms/step - loss: 1.2294 - acc: 0.4979
Epoch 7/30
8/8 [==============================] - 1s 138ms/step - loss: 1.2702 - acc: 0.4627
Epoch 8/30
8/8 [==============================] - 1s 138ms/step - loss: 1.1443 - acc: 0.5434
Epoch 9/30
8/8 [==============================] - 1s 137ms/step - loss: 1.1580 - acc: 0.5552
Epoch 10/30
8/8 [==============================] - 1s 146ms/step - loss: 1.0679 - acc: 0.6172
Epoch 11/30
8/8 [==============================] - 1s 135ms/step - lo

### Testing code:
Here input the required directory path to 'path_test', where the test images are located.
It is assumed that these images have been preprocessed in using Module 2.

In [34]:
# Testing :
path_test = '/home/ignitarium/Desktop/Text_seg/Tester/'
tester = [cv2.imread(file) for file in glob.glob(path_test+'*.jpg')]

In [35]:
# Slightly more preprocessing :
for idx, img in enumerate(tester):
    tester[idx] = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
t = np.array(tester)
t = np.expand_dims(t, axis = 3)

t = t.astype('float32')
t /= 255

### Predictions and Reading them:
The following line prints the predictions of the images inputed into the tester. Note that the images order in the directory maybe different from that of the order that has been read out of the directory.
Hence for verification purposes, one can use plt.imshow(img for img in tester) in the previous snippet.
Here, in each line of output, the highest probability index corresponds to the class that has been predicted:

In [27]:
# Predicting the model's output:
print model.predict(t)

[[1.9698830e-03 1.7484095e-05 9.9643672e-01 1.8797712e-06 9.6719712e-04
  6.0689624e-04]
 [9.9983394e-01 6.2196949e-07 1.5029967e-04 7.3206898e-09 8.5121492e-06
  6.6664516e-06]
 [1.0910032e-06 7.0808419e-05 6.0774516e-07 9.9992633e-01 1.1970848e-06
  5.1995044e-09]
 [2.0270981e-02 1.4248400e-02 6.4458437e-02 1.2899865e-03 5.8559664e-02
  8.4117252e-01]
 [2.3050036e-04 9.9886799e-01 2.9538601e-06 4.9540249e-04 3.7400646e-04
  2.9186158e-05]
 [2.7551632e-03 7.7018780e-03 1.5053352e-02 9.4450833e-03 9.6328408e-01
  1.7605454e-03]]
